# Identifica Stringa_v2.0 

(Non bisongna fare l'explode)

In [2]:
# Specificare la cartella da esplorare
cartella_da_esplorare = r"C:\Miguel\progetti\05_Modifica Testi\01_data\TEST\COPIA"
estensione_file = ".sas"

# Specificare il percorso del file Excel di output
excel_file_path = r"C:\Miguel\progetti\05_Modifica Testi\02_output\ReportPrincipale_20231219_v2.0.xlsx"


In [3]:
import os
import pandas as pd

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)

            # CONDIZIONE
            if all(parola.lower() in line_senza_commenti.lower() for parola in parole_da_verificare) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                root = os.path.dirname(file.name)
                linee_con_parole['Nome File'].append(file_nome)
                linee_con_parole['Percorso File'].append(root)
                linee_con_parole['Riga'].append(line_originale.lower())
                linee_con_parole['Indice'].append(numero)

    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, parole_da_verificare, parole_da_escludere):
    stringa_verificare = ','.join(parole_da_verificare)
    stringa_escludere = ','.join(parole_da_escludere)
    if len(parole_da_escludere) == 0:
        nome_colonna=stringa_verificare
    else:
        nome_colonna=stringa_verificare + ' EXCLUDE ' + stringa_escludere
    
    dati_temporanei = {'Nome File': [], 'Percorso File' : [], nome_colonna: []}
    linee_con_parole = {'Nome File': [], 'Percorso File' : [], 'Riga' :  [], 'Indice': []}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole)
                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei['Percorso File'].append(root)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)

    df_linee = pd.DataFrame(linee_con_parole)

    df_linee['Frequenza Programmi'] = df_linee.groupby('Riga')['Riga'].transform('count')
    df_linee.sort_values(by='Frequenza Programmi',ascending=False, inplace=True)
    df_linee.rename(columns = {'Riga':nome_colonna}, inplace=True)
    
    return dataframe, df_linee

### Creazione Report

In [4]:
# Chiama la funzione per creare il DataFrame
connect_to_pgm, connect_to_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['connect to'], [])

proc_import_pgm, proc_import_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc import'],[])

proc_export_pgm, proc_export_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc export'],[])

mail_pgm, mail_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%email'],[])

let_pgm, let_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%let'],[])

call_symput_pgm, call_symput_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['call symput'],[])

include_pgm, include_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%include'],[])

libname_pgm, libname_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['libname'],["%include"])

# report = pd.concat([connect_to_pgm,  proc_import_pgm, proc_export_pgm, mail_pgm, let_pgm, call_symput_pgm, include_pgm, libname_pgm], axis=1)
# report.to_csv(r'C:\Miguel\progetti\05_Modifica Testi\02_output\Report Catena_20-11-2023\report1.csv')

Frequenza %let

In [5]:
let_linee

,Nome File,Percorso File,%let,Indice,Frequenza Programmi
0,0878_DRI_Ins_Flash_Mao_Rone_Daily_v9.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let ambiente=prd;,14,62
64,PRD_1119_REP_Lst_Chk_Log_Breakdown_v1.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let ambiente=prd;,11,62
198,PRD_1270_DRI_Lst_Listing_Errori_EuroTax_v5.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let ambiente=prd;,10,62
196,PRD_1212_DRI_Lst_Lease_Vehicle_Cus_Daily_v1.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let ambiente=prd;,10,62
192,PRD_1128_DRI_Lst_DRI_Progetto_EDM_EIM_v2.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let ambiente=prd;,9,62
...,...,...,...,...,...
178,0878_Calcolo_Rone_v21.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let cri=ord /* stk ord del*/;,11,1
179,0878_Calcolo_Rone_v21.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let m= mao /*stock mao map*/;,14,1
180,0878_Calcolo_Rone_v21.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let anno_mese=202301/*202208*/;,18,1
181,0878_Calcolo_Rone_v21.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\P...,%let mese=01/*08*/;,20,1


In [6]:
# Frequenza dei nomi delle macro-variabili
name_let = let_linee['%let'].apply(lambda x: x.split('=')[0])
name_let = name_let.apply(lambda x: x.replace('%let ', ''))
name_let = name_let.str.strip().str.replace('\\s+', ' ')
name_let.value_counts()

%let
ambiente              69
jobname               67
path_file_mail_std    16
nomefile               8
*ambiente              7
                      ..
nomefile_03            1
mm_retain              1
nomefile_02            1
n_invoice_limit        1
mese                   1
Name: count, Length: 116, dtype: int64

In [7]:
# let_linee
# call_symput_linee
# include_linee
# libname_linee

# Identifica Stringhe con le regex

In [5]:
import os
import pandas as pd
import re

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # Continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)
            # regex_pattern = r'\bdata\b\s+\S+\.\S+\s*;'
            
            # Verifica la presenza di tutte le parole nella lista e l'assenza delle parole da escludere
            if re.search(regex_pattern, line_senza_commenti.lower(), re.IGNORECASE) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                root = os.path.dirname(file.name)
                
                linee_con_parole['Nome File'].append(file_nome)
                linee_con_parole['Percorso File'].append(root)
                linee_con_parole['Riga'].append(line_originale.lower())
                linee_con_parole['Indice'].append(numero)
              
    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, nome_colonna, regex_pattern, parole_da_escludere):
 
    nome_colonna=nome_colonna
    dati_temporanei = {'Nome File': [], 'Percorso File' : [], nome_colonna: []}
    linee_con_parole = {'Nome File': [], 'Percorso File' : [], 'Riga' :  [], 'Indice': []}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole)
                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei['Percorso File'].append(root)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)

    df_linee = pd.DataFrame(linee_con_parole)
    df_linee['Frequenza Programmi'] = df_linee.groupby('Riga')['Riga'].transform('count')
    df_linee.sort_values(by='Frequenza Programmi',ascending=False, inplace=True)
    df_linee.rename(columns = {'Riga':nome_colonna}, inplace=True)

    return dataframe, df_linee

### Dove scrive...

#### DATA

In [6]:
DATA_pgm, DATA_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement DATA=", r'\bdata\b\s+(\S+\.\S+(?:\s+\S+)*)\s*;',["work."])

In [6]:
DATA_linee

,Nome File,Percorso File,Statement DATA=,Indice,Frequenza Programmi
422,0513_DRI_Ema_BD_PER_SAFO_v35.1.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data xml1.acq_finale;,1226,8
450,SAFO_MEP_v10.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data safo.a_acquisitions_old;,31,8
455,SAFO_RECUPERO_FINE_GIORNATA_STATUS_DATE.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data xml3.opt_finale;,1029,8
457,SAFO_RECUPERO_FINE_GIORNATA_STATUS_DATE.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data xml4.bbx_finale;,1226,8
460,TEST_SAFO_MEP_v10.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data safo.a_acquisitions_old;,220,8
...,...,...,...,...,...
545,1091_SAT_DWH_LIBROMATRICOLA_INS.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data dwh_&ambiente_dwh..sat_dwh_libromatricola...,167,1
544,1091_SAT_DWH_CLAIM_CODES_PRIOR.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data dwh_&ambiente_dwh..sat_dwh_claim_codes_pr...,14,1
543,1091_ASS_Ins_Inserimento_DWH_Satellite_v5.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data dwh_&ambiente..sat_dwh_libromatricola;,413,1
542,1086_DRI_Ins_DB_Sales_Renewals_v10.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data dwh_&ambiente..&table_exp.;,893,1


#### CREATE TABLE

In [12]:
CREATE_TABLE_pgm, CREATE_TABLE_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement CREATE TABLE=", r'\bcreate table\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [13]:
CREATE_TABLE_linee

,Nome File,Percorso File,Statement CREATE TABLE=,Indice,Frequenza Programmi
1110,1248_Controlli_COGE_FUEL_v1.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,"create table save.sheets (num num, nome char(1...",64,25
1033,1204_Controlli_COGE_GL_v2.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,"create table save.sheets (num num, nome char(1...",62,25
998,1203_Controlli_COGE_CAR_v1.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,"create table save.sheets (num num, nome char(1...",64,25
1180,1285_Controlli_COGE_Attive_v3.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,"create table save.sheets (num num, nome char(1...",65,25
1139,1248_Controlli_COGE_FUEL_v5.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,"create table save.sheets (num num, nome char(1...",64,25
...,...,...,...,...,...
591,0953_DRI_Ins_ECU_Enel_Pneumatici_v4.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table bip_&ambiente..sas_ecu_enel_pneu as,536,1
592,0961_BOND_Loader_Item_Log_v1.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table save.qry_result as,167,1
593,0979_DRI_Ins_Company_Credit_Check_Values_v2.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table dwh_&ambiente..dri_dwh_company_cr...,55,1
597,0988_DRI_Ins_Redditivita_Quotazioni_v17.2.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table ammo.ro_varlist as,92,1


In [36]:
FROM_pgm, FROM_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\bfrom\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [37]:
INNER_JOIN_pgm, INNER_JOIN_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\binner join\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [38]:
LEFT_JOIN_pgm, LEFT_JOIN_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\bleft join\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [39]:
SET_pgm, SET_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=",r'\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*',["work."])

In [45]:
x = pd.concat([FROM_linee, INNER_JOIN_linee, LEFT_JOIN_linee, SET_linee])[["Statement="]].drop_duplicates()

In [46]:
x['Statement='].str.extractall(r'\b(dri_dwh\w*)\b')[0].tolist()

['dri_dwh_cr_suppliers_ccf',
 'dri_dwh_customers',
 'dri_dwh_customers',
 'dri_dwh_bol_art94_hist',
 'dri_dwh_suppliers',
 'dri_dwh_customers',
 'dri_dwh_acquisitions',
 'dri_dwh_cus_invoice_details',
 'dri_dwh_dati_mn',
 'dri_dwh_incident_item',
 'dri_dwh_lease',
 'dri_dwh_stato_parco',
 'dri_dwh_lease',
 'dri_dwh_vehicles',
 'dri_dwh_incident_item_log',
 'dri_dwh_bol_art94_hist_ko',
 'dri_dwh_incident_item_joblocatio',
 'dri_dwh_lease_novations_history',
 'dri_dwh_lease',
 'dri_dwh_customers',
 'dri_dwh_contract_management_det',
 'dri_dwh_work_auth',
 'dri_dwh_lease_status_history',
 'dri_dwh_rent_in_charges_payments',
 'dri_dwh_lease_status_history',
 'dri_dwh_activities',
 'dri_dwh_rental_inward',
 'dri_dwh_lease_charges',
 'dri_dwh_lease_novations_history',
 'dri_dwh_lease',
 'dri_dwh_acquisitions_stat_hist',
 'dri_dwh_employees',
 'dri_dwh_suppliers',
 'dri_dwh_quotation_cover_risks',
 'dri_dwh_customers',
 'dri_dwh_risk_req_headers',
 'dri_dwh_acquisitions_stat_hist',
 'dri_dwh_

In [31]:
parole_contenenti_dri_dwh = FROM_linee['Statement FROM='].str.extractall(r'\b(dri_dwh\w*)\b')[0].tolist()

parole_contenenti_dri_dwh

['dri_dwh_cr_suppliers_ccf',
 'dri_dwh_customers',
 'dri_dwh_customers',
 'dri_dwh_bol_art94_hist',
 'dri_dwh_suppliers',
 'dri_dwh_customers',
 'dri_dwh_acquisitions',
 'dri_dwh_cus_invoice_details',
 'dri_dwh_dati_mn',
 'dri_dwh_incident_item',
 'dri_dwh_lease',
 'dri_dwh_stato_parco',
 'dri_dwh_lease',
 'dri_dwh_vehicles',
 'dri_dwh_incident_item_log',
 'dri_dwh_bol_art94_hist_ko',
 'dri_dwh_incident_item_joblocatio',
 'dri_dwh_lease_novations_history',
 'dri_dwh_lease',
 'dri_dwh_customers',
 'dri_dwh_contract_management_det',
 'dri_dwh_work_auth',
 'dri_dwh_lease_status_history',
 'dri_dwh_rent_in_charges_payments',
 'dri_dwh_lease_status_history',
 'dri_dwh_activities',
 'dri_dwh_rental_inward',
 'dri_dwh_lease_charges',
 'dri_dwh_lease_novations_history',
 'dri_dwh_lease',
 'dri_dwh_acquisitions_stat_hist',
 'dri_dwh_employees',
 'dri_dwh_suppliers',
 'dri_dwh_quotation_cover_risks',
 'dri_dwh_customers',
 'dri_dwh_risk_req_headers',
 'dri_dwh_acquisitions_stat_hist',
 'dri_dwh_

### Dove legge

#### SET

In [8]:
SET_pgm, SET_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement SET=",r'\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*',["work."])

In [10]:
SET_linee

,Nome File,Percorso File,Statement SET=,Indice,Frequenza Programmi
878,1400_DRI_Ins_DWH_BOL_FL_Immatricolazioni_v13.0...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data data_sas;set dwh_&ambiente..dri_dwh_bol_f...,59,37
985,dot-8048_proroghe_fittizie.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data data_sas;set dwh_&ambiente..dri_dwh_bol_f...,420,37
1154,SOSPESO- 1538_DRI_Lst_DWH_Flusso_Bolli_Art94_P...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data data_sas;set dwh_&ambiente..dri_dwh_bol_f...,389,37
1153,SOSPESO- 1538_DRI_Lst_DWH_Flusso_Bolli_Art94_P...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data data_sas;set dwh_&ambiente..dri_dwh_bol_f...,71,37
979,1401_DRI_Ins_DWH_BOL_FL_Trasferimenti_v5_0_wip...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data data_sas;set dwh_&ambiente..dri_dwh_bol_f...,58,37
...,...,...,...,...,...
1134,1511_DRI_Ins_DWH_DOP_Fatturazione_Passiva_v7.0...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set &to_table.;,653,1
1133,1511_DRI_Ins_DWH_DOP_Fatturazione_Passiva_v7.0...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set contabilizzati&anno_cur.;,211,1
1132,1511_DRI_Ins_DWH_DOP_Fatturazione_Passiva_v7.0...,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set &tipo.;,146,1
1131,1510_DRI_Ins_DWH_DOP_Fatturazione_Attiva_v4.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set odss_&ambiente..dri_dwh_dop_fatturazione;,361,1


### SAVE ALL

In [9]:
report_all = pd.concat([connect_to_pgm,  proc_import_pgm, proc_export_pgm, mail_pgm, let_pgm, call_symput_pgm, include_pgm, libname_pgm,
            DATA_pgm, SET_pgm
          ], axis=1)

report_all  = report_all.loc[:,~report_all.columns.duplicated()]


In [10]:
report_length = report_all.map(lambda x: len(x) if isinstance(x, list) else x)
totali_per_colonna = report_all.drop(columns='Percorso File').map(lambda x: len(x) if isinstance(x, list) else 0).sum()
conteggi_per_colonna = (report_all.drop(columns='Percorso File').map(lambda x: len(x) if isinstance(x, list) else 0) != 0).sum()

report_means = pd.DataFrame({
    'Keyword': totali_per_colonna.index,
    'Numero Programmi' : report_all.shape[0],
    'Frequenza Totale': totali_per_colonna.values,
    'Frequenza Programmi' : conteggi_per_colonna.values,
    'Percentuale Programmi' : conteggi_per_colonna.values/report_all.shape[0]
})

In [13]:
report_means

,Keyword,Numero Programmi,Frequenza Totale,Frequenza Programmi,Percentuale Programmi
0,connect to,1002,914,368,0.367265
1,proc import,1002,1042,338,0.337325
2,proc export,1002,1839,569,0.567864
3,%email,1002,513,460,0.459082
4,%let,1002,4679,872,0.870259
5,call symput,1002,3538,690,0.688623
6,%include,1002,5374,817,0.815369
7,libname EXCLUDE %include,1002,1491,759,0.757485
8,Statement DATA=,1002,857,262,0.261477
9,Statement SET=,1002,1364,386,0.385230


In [11]:
import pandas as pd

# Creare un oggetto ExcelWriter
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    # Salvare i DataFrame nei fogli desiderati
    report_means.to_excel(writer, sheet_name='COVER', index=False)
    report_all.reset_index().to_excel(writer, sheet_name='Lista PGM_v1.0', index=False)
    report_length.reset_index().to_excel(writer, sheet_name='Lista PGM_v2.0', index=False)
    DATA_linee.to_excel(writer, sheet_name='OUTPUT', index=False)
    CREATE_TABLE_linee.to_excel(writer, sheet_name='CREATE TABLE', index=False)
    SET_linee.to_excel(writer, sheet_name='INPUT', index=False)
    
    let_linee.to_excel(writer, sheet_name='%LET', index=False)
    call_symput_linee.to_excel(writer, sheet_name='CALL SYMPUT', index=False)
    include_linee.to_excel(writer, sheet_name='INCLUDE', index=False)
    libname_linee.to_excel(writer, sheet_name='LIBNAME', index=False)